In [2]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [4]:
df = pd.read_csv("/content/NPHA-doctor-visits.csv")

df = df.dropna()


X = df.drop('Number of Doctors Visited', axis=1)
y = df['Number of Doctors Visited']


for col in X.select_dtypes(include='object').columns:
    X[col] = LabelEncoder().fit_transform(X[col])


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)


X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)


class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(X.shape[1], 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        return self.layers(x)

model = SimpleNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train model
for epoch in range(100):
    model.train()
    output = model(X_train)
    loss = criterion(output, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

model.eval()
with torch.no_grad():
    test_output = model(X_test)
    test_loss = criterion(test_output, y_test)
    print(f"\nTest MSE: {test_loss.item():.4f}")

Epoch 10, Loss: 2.9818
Epoch 20, Loss: 1.1179
Epoch 30, Loss: 0.7616
Epoch 40, Loss: 0.4928
Epoch 50, Loss: 0.4675
Epoch 60, Loss: 0.4437
Epoch 70, Loss: 0.4227
Epoch 80, Loss: 0.4110
Epoch 90, Loss: 0.4035
Epoch 100, Loss: 0.3969

Test MSE: 0.5162
